In [1]:
import os
import sys

from typing import List, Tuple
from pandas import read_csv
import pandas as pd
import random
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput
from bokeh.plotting import figure as bokeh_figure
from os.path import dirname, join
from bokeh.models import HelpTool, HoverTool
from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap
from bokeh.transform import cumsum
from bokeh.models import FactorRange
from bokeh.transform import dodge
from bokeh.models import Panel, Tabs
from bokeh.layouts import column, row
from bokeh.models import Dropdown
from bokeh.palettes import RdYlBu3
from bokeh.palettes import Category20c

from bokeh.plotting import curdoc
from bokeh.models import ColumnDataSource, Div, Select
from bokeh.plotting import figure as bokeh_figure
from bokeh.models import CheckboxButtonGroup, CustomJS
from bokeh.models import MultiChoice
from bokeh.models import Paragraph
from masci_tools.vis.bokeh_plots import bokeh_scatter, bokeh_multi_scatter, bokeh_line
sys.path.append(".")


In [3]:
from analysis import filter_dataframe, calculate_crosstab
#from analysis import prepare_data_research_field
output_notebook()

Loading BokehJS ...

In [4]:
pwd = os.getcwd()
datafilepath_raw = join(pwd, './data/HMC_community_survey2021_merged_all_replies.csv')
datafilepath= join(pwd, './data/20211130_HMCCommSurvey_clean.csv')
surveydata = read_csv(datafilepath)

In [12]:
q1_key = 'careerLevel'
q2_key = 'docStructured'
df1 = calculate_crosstab(surveydata, q1_key, q2_key) 
df1

,careerLevel,docStructured,value,total,percentage
0,Director (of the institute),I am not sure,1,22,4.545455
1,Other,I am not sure,2,65,3.076923
2,PhD student,I am not sure,20,130,15.384615
3,Postdoc,I am not sure,31,128,24.218750
4,Principal Investigator,I am not sure,19,148,12.837838
5,Research associate,I am not sure,27,272,9.926471
6,Undergraduate / Masters student,I am not sure,1,24,4.166667
7,Director (of the institute),No,6,22,27.272727
8,Other,No,13,65,20.000000
9,PhD student,No,32,130,24.615385


In [36]:
list(df1['careerLevel'].value_counts().keys())

['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']

In [13]:
selected = ColumnDataSource(df1)

In [58]:
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter)
from bokeh.transform import transform
from bokeh.models import Legend, Circle

def bokeh_corr_plot(source, x='value', y='counts',  figure=None, title='', x_range=None, y_range=None, 
                    markersize='markersize',  xlabel='Answers', ylabel='Number of answers', alpha=None, **kwargs):
    """Plot an interactive circle with bokeh"""
    
    if figure is None:
        fig = bokeh_figure(height=600, width=600,
               title=title,
               toolbar_location='right',
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    else:
        fig = figure
    if alpha is None:
        alpha = 1.0
    
    # this is the colormap from the original NYTimes plot
    #colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
    #mapper = LinearColorMapper(palette=colors, low=source.data[y].min(), high=source.data[y].max())
    
    fig.circle(source=source, x=x, y=y, radius=0.9, **kwargs)#,# fill_color={'field': y, 'transform': color_mapper}, 
    
    
    #           fill_alpha=alpha)
    #fig.rect(x=x, y=y, width=1, height=1, source=source,
    #   line_color=None, fill_color=transform('percentage', mapper))    
    
    
    #line_color='#7c7e71',
    #line_width=0.5,
    #line_alpha=0.5,
    #legend_group='region')
    
    #fig.y_range.start = 0
    #fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    #fig.xgrid.grid_line_color = None
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel


    #fig.legend.location = "left"
    #fig.legend.orientation = "horizontal"
    #fig.legend.click_policy="hide"

    return fig

In [60]:

y_range= ['No', 'I am not sure', 'Yes']
x_range= ['Director (of the institute)',
 'Other',
 'PhD student',
 'Postdoc',
 'Principal Investigator',
 'Research associate',
 'Undergraduate / Masters student']
fig = bokeh_figure(height=600, width=600,
               title='title',
               toolbar_location='right',
               tools='hover',
                  y_range=y_range, x_range=x_range)

circle = fig.circle(source=selected, x='careerLevel', y='docStructured', size='percentage', alpha=0.6, hover_alpha=1.0, 
           hover_line_color='black', hover_line_width=5)#radius=0.1,
fig.xaxis.major_label_orientation = 1

def create_legend_items(number, size_min, color):
    leg_items = []
    step = 100/number
    size = min_size
    alpha_min = 0.2
    for i in range(number):
        circ = Circle(size=size, fill_color=color[i])
        lege_items.append((f'{size}' [circ]))
        size = size + step
    
    return leg_items


leg_items = create_legend_items(5, size_min=1 #[("circle", [circle])]
legend = Legend(
        title='rel. amount of answers [%]',
        items=leg_items,
        location='top', orientation='vertical',
        border_line_color=None
)
fig.add_layout(legend, 'right')
show(fig)

In [46]:
fig = bokeh_corr_plot(source=selected, x='careerLevel', y='docStructured', markersize='percentage')
show(fig)

ERROR:bokeh.core.validation.check:E-1020 (BAD_EXTRA_RANGE_NAME): An extra range name is configured with a name that does not correspond to any range: x_range_name='careerLevel', y_range_name='docStructured' [GlyphRenderer(id='3052', ...)]
